In [12]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def extract_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    guru_keypoints = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stop when video ends

        # Convert frame to RGB for processing
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            keypoints = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]
            guru_keypoints.append(keypoints)

    cap.release()
    np.save("guru_poses.npy", guru_keypoints)  # Save for future comparisons

# Run the function on your Guru's video
extract_keypoints("/Users/aditinarayan/Desktop/Computer_vision/trimmed_video.mp4")
print("✅ Guru's keypoints saved!")

NameError: name 'audio_classifier' is not defined

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize webcam capture
cap = cv2.VideoCapture(0)

# Track student poses over time
student_keypoints_sequence = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame for a better user experience
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB (MediaPipe expects RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect the student's pose
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        # Extract keypoints (x, y coordinates)
        student_keypoints = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]
        
        # Store the student's keypoints sequence (for DTW)
        student_keypoints_sequence.append(student_keypoints)

        # Display the student's pose landmarks on the frame
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Show the frame
    cv2.imshow("Student vs Guru", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1738565097.749482 1654760 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1738565097.906529 1690724 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738565097.926683 1690727 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-02-03 12:15:47.108 python[54387:1654760] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-03 12:15:47.108 python[54387:1654760] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


KeyboardInterrupt: 

: 

In [15]:
import cv2
import mediapipe as mp
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Initialize Pose Estimation Model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Load Guru’s Pre-recorded Video
guru_video_path = "/Users/aditinarayan/Desktop/Computer_vision/trimmed_video.mp4"
guru_video = cv2.VideoCapture(guru_video_path)

# Load Guru’s Pose Data (Pre-extracted)
guru_poses = np.load("/Users/aditinarayan/Desktop/Computer_vision/guru_poses.npy", allow_pickle=True)

# Initialize Webcam for Student
student_cam = cv2.VideoCapture(0)

# Get Guru video dimensions
guru_width = int(guru_video.get(cv2.CAP_PROP_FRAME_WIDTH))
guru_height = int(guru_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

while student_cam.isOpened() and guru_video.isOpened():
    # Read Guru's video frame
    ret_guru, guru_frame = guru_video.read()
    if not ret_guru:
        guru_video.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Restart Guru’s Video
        continue

    # Read Student's live video frame
    ret_student, student_frame = student_cam.read()
    if not ret_student:
        break

    # Convert student frame to RGB (for MediaPipe)
    rgb_student_frame = cv2.cvtColor(student_frame, cv2.COLOR_BGR2RGB)
    student_results = pose.process(rgb_student_frame)

    # Extract student keypoints if detected
    student_keypoints = None
    if student_results.pose_landmarks:
        student_keypoints = [(lm.x, lm.y) for lm in student_results.pose_landmarks.landmark]

        # Ensure Guru & Student Keypoints are of same length before DTW
        if len(student_keypoints) == len(guru_poses[0]):
            distance, _ = fastdtw(student_keypoints, guru_poses[0], dist=euclidean)
            similarity_score = max(0, 100 - distance)  # Normalize to 0-100%
        else:
            similarity_score = 0  # Default score if shape mismatch

        # Draw Student's pose landmarks
        mp_drawing.draw_landmarks(student_frame, student_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display Score
        cv2.putText(student_frame, f"Score: {int(similarity_score)}%", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Resize both frames to match Guru’s video size
    student_frame = cv2.resize(student_frame, (guru_width, guru_height))
    guru_frame = cv2.resize(guru_frame, (guru_width, guru_height))

    # Combine the two frames horizontally
    combined_frame = np.hstack((guru_frame, student_frame))

    # Show the combined frame
    cv2.imshow("Guru vs Student", combined_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
student_cam.release()
guru_video.release()
cv2.destroyAllWindows()


NameError: name 'audio_classifier' is not defined